In [ ]:
!pip install -Uqq duckduckgo_search
!pip install -Uqq fastai
!pip install -Uqq fastbook

In [ ]:
from duckduckgo_search import ddg_images
from fastcore.all import *
from fastai.vision.all import *

from fastdownload import download_url

# Data

In [ ]:
def search_images(term, max_images=200): 
    return L(ddg_images(term, max_results=max_images)).itemgot('image')

In [ ]:
urls = search_images('warka', max_images=1)

In [ ]:
dest = 'warka.jpg'
download_url(urls[0], dest, show_progress=False)

im = Image.open(dest)
im.to_thumb(256, 256)

In [ ]:
searches = 'warka', 'harnas', 'perla'
im_path = Path('beers')
from time import sleep

for o in searches:
    dest = (im_path/o)
    dest.mkdir(exist_ok=True, parents=True)
    download_images(dest, urls=search_images(f'piwo {o}', max_images=20))
    sleep(10)  # Pause between searches to avoid over-loading server

In [ ]:
fns = get_image_files(im_path)
fns

In [ ]:
failed = verify_images(fns)
failed

In [ ]:
failed.map(Path.unlink);

In [ ]:
??verify_images

In [ ]:
beers = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

In [ ]:
dls = beers.dataloaders(im_path)

In [ ]:
dls.valid.show_batch(max_n=4, nrows=1)

In [ ]:
beers = beers.new(item_tfms=Resize(128, ResizeMethod.Squish))
dls = beers.dataloaders(im_path)
dls.valid.show_batch(max_n=4, nrows=1)

In [ ]:
beers = beers.new(item_tfms=Resize(128, ResizeMethod.Pad, pad_mode='zeros'))
dls = beers.dataloaders(im_path)
dls.valid.show_batch(max_n=4, nrows=1)

In [ ]:
beers = beers.new(item_tfms=Resize(128), batch_tfms=aug_transforms(mult=2))
dls = beers.dataloaders(im_path, bs=10)
dls.train.show_batch(max_n=8, nrows=2, unique=True)

# Training

In [ ]:
beers = beers.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms())
dls = beers.dataloaders(im_path, bs=16)

In [ ]:
learn = vision_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(10)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

# Clean & Fine tune

In [ ]:
interp.plot_top_losses(5, nrows=1)

In [ ]:
from fastai.vision.widgets import ImageClassifierCleaner
cleaner = ImageClassifierCleaner(learn)
cleaner

In [ ]:
for idx in cleaner.delete(): cleaner.fns[idx].unlink()

In [ ]:
for idx,cat in cleaner.change(): shutil.move(str(cleaner.fns[idx]), path/cat)


# Inference

In [ ]:
learn.export()

In [ ]:
path = Path()
path.ls(file_exts='.pkl')

In [ ]:
learn_inf = load_learner(path/'export.pkl')

In [ ]:
learn_inf.predict('harnas.jpg')

In [ ]:
learn_inf.dls.vocab

# Notebook app

In [ ]:
import ipywidgets as widgets

btn_upload = widgets.FileUpload()
btn_upload

In [ ]:
img = PILImage.create(btn_upload.data[-1])

In [ ]:
out_pl = widgets.Output()
out_pl.clear_output()
with out_pl: display(img.to_thumb(128,128))
out_pl

In [ ]:
pred,pred_idx,probs = learn_inf.predict(img)

In [ ]:
lbl_pred = widgets.Label()
lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'
lbl_pred

In [ ]:
btn_run = widgets.Button(description='Classify')
btn_run

In [ ]:
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

btn_run.on_click(on_click_classify)

In [ ]:
btn_upload = widgets.FileUpload()

In [ ]:
from ipywidgets import VBox

VBox([widgets.Label('Select your bear!'), 
      btn_upload, btn_run, out_pl, lbl_pred])